## tf function和auto graph

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.0
numpy 1.14.5
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


### tf.function

使用tf.function可以将普通的python函数转换为tensorflow的图结构，从而提高代码运行效率     

1.显示转换方式

In [2]:
# 先定义一个普通的python函数 
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    # 在tf中，三元表达式用where实现
    return scale * tf.where(is_positive, z, scale * alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3., -2.5])))

# 显示转换为tf.function 
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3., -2.5])))

print(scaled_elu_tf.python_function is scaled_elu)

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
True


测试一下两个函数的执行效率

In [6]:
# 使用1000000的大矩阵进行测试
%timeit scaled_elu(tf.random.normal((1000, 1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

7.22 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.66 ms ± 96 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


2.使用annotation方式进行转换

In [7]:
# 计算1 + 1/2 + 1/2^2 + ... + 1/2^n

@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


In [9]:
# 输出函数转换过程的中间代码
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

display_tf_code(scaled_elu)

```python
from __future__ import print_function

def tf__scaled_elu(z, scale=None, alpha=None):
  try:
    with ag__.function_scope('scaled_elu'):
      do_return = False
      retval_ = None
      is_positive = ag__.converted_call('greater_equal', tf, ag__.ConversionOptions(recursive=True, verbose=0, strip_decorators=(ag__.convert, ag__.do_not_convert, ag__.converted_call), force_conversion=False, optional_features=ag__.Feature.ALL, internal_convert_user_code=True), (z, 0.0), {})
      do_return = True
      retval_ = scale * ag__.converted_call('where', tf, ag__.ConversionOptions(recursive=True, verbose=0, strip_decorators=(ag__.convert, ag__.do_not_convert, ag__.converted_call), force_conversion=False, optional_features=ag__.Feature.ALL, internal_convert_user_code=True), (is_positive, z, scale * alpha * tf.nn.elu(z)), {})
      return retval_
  except:
    ag__.rewrite_graph_construction_error(ag_source_map__)



tf__scaled_elu.autograph_info__ = {}

```

### 在tf.function中使用variable 
在使用tf.function对某个python函数进行转换时，需要保证函数体内没有对variable的定义,否则会抛出异常     

In [12]:
var = tf.Variable(0.)

@tf.function
def add_20():
    return var.assign_add(20)

print(add_20())

tf.Tensor(20.0, shape=(), dtype=float32)


### tf.function.ConcreteFunction    
使用tf.function对一个python函数进行转换后，可以通过input_signature对输入参数的类型进行限定   

In [15]:
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)

try:
    # 如果输入参数类型与input_signature不符，会抛出异常
    print(cube(tf.constant(1., 2., 3.)))
except ValueError as ex:
    print(ex)
print(cube(tf.constant([1, 2, 3])))

Python inputs incompatible with input_signature: inputs ((<tf.Tensor: id=53745, shape=(3,), dtype=float64, numpy=array([1., 1., 1.])>,)), input_signature ((TensorSpec(shape=(None,), dtype=tf.int32, name='x'),))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


使用get_concrete_function转换为可以保存的savedmodel形式   
可以从中获取各个结点信息和op的信息   

In [18]:
# @tf.function py func -> tf graph
# get_concrete_function -> add input_signature -> savedModel
# 获取ConcreteFunction对象
cube_func_int32 = cube.get_concrete_function(
    tf.TensorSpec([None], tf.int32))
print(cube_func_int32)

In [19]:
# 从ConcreteFunction中获取图结构及相关信息   
cube_func_int32.graph

In [20]:
# 获取图结构中的op
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [21]:
# 打印当前的图结构信息 
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 27
}

In [22]:
# 通过name获取op或者tensor，这个name可以在定义时指定
cube_func_int32.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [23]:
cube_func_int32.graph.get_tensor_by_name('x:0')

<tf.Tensor 'x:0' shape=(None,) dtype=int32>